# Data Transformation

## Setup

In [1]:
# Setting up execution path
import os

print(f"Current working directory: {os.path.basename(os.getcwd())}")

# Change to root directory
os.chdir("../")
print(f"Current working directory (Changed): {os.path.basename(os.getcwd())}")

Current working directory: notebooks
Current working directory (Changed): Analyzing-Pokemons


In [2]:
from rich import print

In [3]:
from src.constants import CONFIGS
from src.utils.basic_utils import read_yaml

In [4]:
configs = read_yaml(CONFIGS).data_preprocessor
print(configs.to_dict())

[2024-02-26 12:45:44 PM]:ProjectLogger INFO:basic_utils 40 - yaml file: conf\configs.yaml loaded successfully


{'scraped_data_path': 'data/external/scraped_pokemons.csv', 'processed_data_path': 'data/processed/pokemons.csv'}

In [5]:
#imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# module setup
%matplotlib inline
pd.options.display.precision = 5
warnings.filterwarnings("ignore")

In [6]:
# Get the CSV filepath from configs
scraped_data_path = configs.scraped_data_path

# Read the CSV data
pokemons_df_main = pd.read_csv(scraped_data_path, index_col=False)

# create a copy of it for working
df = pokemons_df_main.copy(deep=True)

# View a glimpse of the data
df.head()

,rank,name,types,total_power,hit_points,attack,defense,special_attack,special_defense,speed,icon_url,details_url,scrape_ts
0,1,Bulbasaur,"Grass, Poison",318,45,49,49,65,65,45,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/bulbasaur,2024-02-24 15:05:21
1,2,Ivysaur,"Grass, Poison",405,60,62,63,80,80,60,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/ivysaur,2024-02-24 15:05:21
2,3,Venusaur,"Grass, Poison",525,80,82,83,100,100,80,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/venusaur,2024-02-24 15:05:21
3,3,Mega Venusaur,"Grass, Poison",625,80,100,123,122,120,80,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/venusaur,2024-02-24 15:05:21
4,4,Charmander,Fire,309,39,52,43,60,50,65,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/charmander,2024-02-24 15:05:21


In [7]:
# Create boolean mask to get duplicated entries
dup_mask = df.duplicated()

# Apply the mask over dataframe
dup_entries = df[dup_mask]

# Print the results
print(f"There are {len(dup_entries)} duplicated entries in the dataframe")

There are 0 duplicated entries in the dataframe

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1215 entries, 0 to 1214
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   rank             1215 non-null   int64 
 1   name             1215 non-null   object
 2   types            1215 non-null   object
 3   total_power      1215 non-null   int64 
 4   hit_points       1215 non-null   int64 
 5   attack           1215 non-null   int64 
 6   defense          1215 non-null   int64 
 7   special_attack   1215 non-null   int64 
 8   special_defense  1215 non-null   int64 
 9   speed            1215 non-null   int64 
 10  icon_url         1215 non-null   object
 11  details_url      1215 non-null   object
 12  scrape_ts        1215 non-null   object
dtypes: int64(8), object(5)
memory usage: 123.5+ KB


In [9]:
for col in df.columns:
    unq_count = df[col].nunique()
    print(f"The unique values in {col} column is {unq_count}")

The unique values in rank column is 1025

The unique values in name column is 1193

The unique values in types column is 221

The unique values in total_power column is 228

The unique values in hit_points column is 109

...

The unique values in special_defense column is 107

The unique values in speed column is 127

The unique values in icon_url column is 1215

The unique values in details_url column is 1025

The unique values in scrape_ts column is 2

As we can see expect the "icon_url" column, none of the other columns has a complete 
unique values.

Therefore, we need a primary key to identify each row individually and we can call it "pokemon_id"

In [10]:
df["calculated_power"] = (
    df["attack"]
    + df["defense"]
    + df["special_attack"]
    + df["special_defense"]
    + df["speed"]
    + df["hit_points"]
)

df.head()

,rank,name,types,total_power,hit_points,attack,defense,special_attack,special_defense,speed,icon_url,details_url,scrape_ts,calculated_power
0,1,Bulbasaur,"Grass, Poison",318,45,49,49,65,65,45,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/bulbasaur,2024-02-24 15:05:21,318
1,2,Ivysaur,"Grass, Poison",405,60,62,63,80,80,60,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/ivysaur,2024-02-24 15:05:21,405
2,3,Venusaur,"Grass, Poison",525,80,82,83,100,100,80,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/venusaur,2024-02-24 15:05:21,525
3,3,Mega Venusaur,"Grass, Poison",625,80,100,123,122,120,80,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/venusaur,2024-02-24 15:05:21,625
4,4,Charmander,Fire,309,39,52,43,60,50,65,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/charmander,2024-02-24 15:05:21,309


In [11]:
df[df["calculated_power"] != df["total_power"]]

,rank,name,types,total_power,hit_points,attack,defense,special_attack,special_defense,speed,icon_url,details_url,scrape_ts,calculated_power


In [12]:
df = df.drop(columns="calculated_power")

As we can notice that the `total_power` is nothing but the sum of all the stats, so we can remove this column. 

Other transformation we need in the dataset are as follows:

1. Create a unique row id: "pokemon_id"
2. Remove the "total_power" column
3. Split the "types" into rows
4. Unpivot the metrics column
5. Create a "pokemon.db" sqlite database
6. Create a table `pokemons` with the following columns:
   - "pokemon_id", "rank", "name", "icon_url", "details_url" , "scrape_ts" and remove duplicates
7. Create a table `stats` with the following columns:
   - "pokemon_id", "types", "metric", "value". This table needs to have a row_id (auto_incemented)


### Unique Row ID Addition

In [13]:
custom_index_col = pd.RangeIndex(
    start=1000, stop=1000 + len(df), step=1, name="pokemon_id"
)



df.index = custom_index_col

df.index = "P" + df.index.astype("string")

df = df.reset_index()

In [14]:
df.head()

,pokemon_id,rank,name,types,total_power,hit_points,attack,defense,special_attack,special_defense,speed,icon_url,details_url,scrape_ts
0,P1000,1,Bulbasaur,"Grass, Poison",318,45,49,49,65,65,45,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/bulbasaur,2024-02-24 15:05:21
1,P1001,2,Ivysaur,"Grass, Poison",405,60,62,63,80,80,60,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/ivysaur,2024-02-24 15:05:21
2,P1002,3,Venusaur,"Grass, Poison",525,80,82,83,100,100,80,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/venusaur,2024-02-24 15:05:21
3,P1003,3,Mega Venusaur,"Grass, Poison",625,80,100,123,122,120,80,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/venusaur,2024-02-24 15:05:21
4,P1004,4,Charmander,Fire,309,39,52,43,60,50,65,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/charmander,2024-02-24 15:05:21


In [15]:
df["types"] = df["types"].str.split(",")

In [16]:
# Explode the 'types_2' column
df_exploded = df.explode("types")
df_exploded.head()

,pokemon_id,rank,name,types,total_power,hit_points,attack,defense,special_attack,special_defense,speed,icon_url,details_url,scrape_ts
0,P1000,1,Bulbasaur,Grass,318,45,49,49,65,65,45,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/bulbasaur,2024-02-24 15:05:21
0,P1000,1,Bulbasaur,Poison,318,45,49,49,65,65,45,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/bulbasaur,2024-02-24 15:05:21
1,P1001,2,Ivysaur,Grass,405,60,62,63,80,80,60,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/ivysaur,2024-02-24 15:05:21
1,P1001,2,Ivysaur,Poison,405,60,62,63,80,80,60,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/ivysaur,2024-02-24 15:05:21
2,P1002,3,Venusaur,Grass,525,80,82,83,100,100,80,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/venusaur,2024-02-24 15:05:21


In [17]:
df_exploded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1884 entries, 0 to 1214
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   pokemon_id       1884 non-null   string
 1   rank             1884 non-null   int64 
 2   name             1884 non-null   object
 3   types            1884 non-null   object
 4   total_power      1884 non-null   int64 
 5   hit_points       1884 non-null   int64 
 6   attack           1884 non-null   int64 
 7   defense          1884 non-null   int64 
 8   special_attack   1884 non-null   int64 
 9   special_defense  1884 non-null   int64 
 10  speed            1884 non-null   int64 
 11  icon_url         1884 non-null   object
 12  details_url      1884 non-null   object
 13  scrape_ts        1884 non-null   object
dtypes: int64(8), object(5), string(1)
memory usage: 220.8+ KB


### Remove Columns

In [18]:
df_exploded = df_exploded.drop(columns=["total_power"])
df_exploded.head()

,pokemon_id,rank,name,types,hit_points,attack,defense,special_attack,special_defense,speed,icon_url,details_url,scrape_ts
0,P1000,1,Bulbasaur,Grass,45,49,49,65,65,45,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/bulbasaur,2024-02-24 15:05:21
0,P1000,1,Bulbasaur,Poison,45,49,49,65,65,45,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/bulbasaur,2024-02-24 15:05:21
1,P1001,2,Ivysaur,Grass,60,62,63,80,80,60,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/ivysaur,2024-02-24 15:05:21
1,P1001,2,Ivysaur,Poison,60,62,63,80,80,60,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/ivysaur,2024-02-24 15:05:21
2,P1002,3,Venusaur,Grass,80,82,83,100,100,80,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/venusaur,2024-02-24 15:05:21


### Unpivot Stats

In [19]:
df_exploded = df_exploded.melt(
    id_vars=[
        "pokemon_id",
        "rank",
        "name",
        "types",
        "icon_url",
        "details_url",
        "scrape_ts",
    ],
    value_vars=[
        "hit_points",
        "attack",
        "defense",
        "special_attack",
        "special_defense",
        "speed",
    ],
    var_name="metric",
    value_name="value",
)


df_exploded

,pokemon_id,rank,name,types,icon_url,details_url,scrape_ts,metric,value
0,P1000,1,Bulbasaur,Grass,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/bulbasaur,2024-02-24 15:05:21,hit_points,45
1,P1000,1,Bulbasaur,Poison,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/bulbasaur,2024-02-24 15:05:21,hit_points,45
2,P1001,2,Ivysaur,Grass,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/ivysaur,2024-02-24 15:05:21,hit_points,60
3,P1001,2,Ivysaur,Poison,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/ivysaur,2024-02-24 15:05:21,hit_points,60
4,P1002,3,Venusaur,Grass,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/venusaur,2024-02-24 15:05:21,hit_points,80
...,...,...,...,...,...,...,...,...,...
11299,P2211,1024,Normal Form,Normal,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/terapagos,2024-02-24 15:05:22,speed,60
11300,P2212,1024,Terastal Form,Normal,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/terapagos,2024-02-24 15:05:22,speed,85
11301,P2213,1024,Stellar Form,Normal,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/terapagos,2024-02-24 15:05:22,speed,85
11302,P2214,1025,Pecharunt,Poison,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/pecharunt,2024-02-24 15:05:22,speed,88


### Create `pokemons` Table

In [20]:
req_cols = ["pokemon_id", "rank", "name", "icon_url", "details_url"]
pokemons_tbl = df_exploded[req_cols]
pokemons_tbl.head()

,pokemon_id,rank,name,icon_url,details_url
0,P1000,1,Bulbasaur,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/bulbasaur
1,P1000,1,Bulbasaur,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/bulbasaur
2,P1001,2,Ivysaur,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/ivysaur
3,P1001,2,Ivysaur,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/ivysaur
4,P1002,3,Venusaur,https://img.pokemondb.net/sprites/scarlet-viol...,https://pokemondb.net/pokedex/venusaur


In [21]:
pokemons_tbl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11304 entries, 0 to 11303
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   pokemon_id   11304 non-null  string
 1   rank         11304 non-null  int64 
 2   name         11304 non-null  object
 3   icon_url     11304 non-null  object
 4   details_url  11304 non-null  object
dtypes: int64(1), object(3), string(1)
memory usage: 441.7+ KB


In [22]:
pokemons_tbl = pokemons_tbl.drop_duplicates()

In [23]:
stat_tbl_cols = ["pokemon_id", "types", "metric", "value"]
stats_tbl = df_exploded[stat_tbl_cols]
stats_tbl.head()

,pokemon_id,types,metric,value
0,P1000,Grass,hit_points,45
1,P1000,Poison,hit_points,45
2,P1001,Grass,hit_points,60
3,P1001,Poison,hit_points,60
4,P1002,Grass,hit_points,80


In [24]:
stats_tbl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11304 entries, 0 to 11303
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   pokemon_id  11304 non-null  string
 1   types       11304 non-null  object
 2   metric      11304 non-null  object
 3   value       11304 non-null  int64 
dtypes: int64(1), object(2), string(1)
memory usage: 353.4+ KB


In [25]:
stats_tbl = stats_tbl.drop_duplicates()

### SQLite Database

In [26]:
import sqlite3

In [27]:
conn = sqlite3.connect("data/pokemon_stats.db")

In [28]:
pokemons_tbl.to_sql("pokemons", conn, index=False, if_exists="replace")

1215

In [29]:
stats_tbl.to_sql("stats", conn, index=False, if_exists="replace")

11304